In [7]:
import rasterio as rio
from rasterio.warp import reproject, Resampling
from rasterio.transform import Affine


In [8]:
path_rsimg = 'data/data-section-6/s2_10m_chenggong.tif'
path_rsimg_resam = 'data/data-section-6/s2_10m_chenggong_resam.tif'


In [9]:
x_resam_res = 21 
y_resam_res = 22 


In [10]:
with rio.open(path_rsimg) as src:
    data = src.read(1)
    src_transform = src.transform
    src_crs = src.crs
    src_width = src.width
    src_height = src.height

src_transform
# data.shape
src_crs


CRS.from_wkt('PROJCS["WGS 84 / UTM zone 48N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",105],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')

In [11]:
## 1. 获取原影像空间分辨率
x_res = src_transform.a
y_res =src_transform.e

## 2. 计算缩放比例
x_scale = x_res/x_resam_res
y_scale = y_res/y_resam_res

## 3. 计算重采样影像尺寸
dst_width = int(src_width*x_scale)
dst_height = int(src_height*y_scale)
print(dst_width, dst_height)

## 4. 重采样影像仿射变换矩阵
dst_transform = Affine(
    a=x_resam_res,
    b=0,
    c=src_transform.c,
    d=0,
    e=-y_resam_res,
    f=src_transform.f
)



1426 -1465


In [12]:
### 重采样并写出

with rio.open(
    fp=path_rsimg_resam,
    mode='w',
    driver='GTiff',
    height=-dst_height,
    width=dst_width,
    count=1,
    dtype=data.dtype,
    crs=src_crs,
    transform=dst_transform) as dst:

    reproject(
        source=data,
        src_transform=src_transform,
        src_crs=src_crs,
        destination=rio.band(dst, 1),
        dst_transform=dst_transform,
        dst_crs=src_crs,
        resampling=Resampling.bilinear
    )




In [13]:
from rasterio.warp import calculate_default_transform
from pyproj import CRS

path_rsimg_reproj = 'data/data-section-6/s2_10m_chenggong_reproj.tif'
reproj_crs = CRS.from_epsg(4326)
reproj_crs = reproj_crs.to_wkt()
reproj_crs


'GEOGCRS["WGS 84",ENSEMBLE["World Geodetic System 1984 ensemble",MEMBER["World Geodetic System 1984 (Transit)"],MEMBER["World Geodetic System 1984 (G730)"],MEMBER["World Geodetic System 1984 (G873)"],MEMBER["World Geodetic System 1984 (G1150)"],MEMBER["World Geodetic System 1984 (G1674)"],MEMBER["World Geodetic System 1984 (G1762)"],MEMBER["World Geodetic System 1984 (G2139)"],MEMBER["World Geodetic System 1984 (G2296)"],ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]],ENSEMBLEACCURACY[2.0]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],CS[ellipsoidal,2],AXIS["geodetic latitude (Lat)",north,ORDER[1],ANGLEUNIT["degree",0.0174532925199433]],AXIS["geodetic longitude (Lon)",east,ORDER[2],ANGLEUNIT["degree",0.0174532925199433]],USAGE[SCOPE["Horizontal component of 3D system."],AREA["World."],BBOX[-90,-180,90,180]],ID["EPSG",4326]]'

In [14]:
with rio.open(path_rsimg) as src:
    data = src.read(1)
    src_transform = src.transform
    src_crs = src.crs
    src_width = src.width
    src_height = src.height
    src_bounds = src.bounds

src_transform
# # data.shape
# # src_cr
# src_bounds


Affine(10.0, 0.0, 268170.0,
       0.0, -10.0, 2765450.0)

In [15]:
dst_transform, dst_width, dst_height = calculate_default_transform(
    src_crs=src_crs,
    dst_crs=reproj_crs,
    width=src.width,
    height=src.height,
    left=src_bounds.left,
    bottom=src_bounds.bottom,
    right=src_bounds.right,
    top=src_bounds.top
)

dst_transform
# dst_height

Affine(9.45063521110585e-05, 0.0, 102.70324341425528,
       0.0, -9.45063521110585e-05, 24.99108870425977)

In [16]:
### 重投影并写出

with rio.open(
    fp=path_rsimg_reproj,
    mode='w',
    driver='GTiff',
    height=dst_height,
    width=dst_width,
    count=1,
    dtype=data.dtype,
    crs=reproj_crs,
    transform=dst_transform) as dst:

    reproject(
        source=data,
        src_transform=src_transform,
        src_crs=src_crs,
        destination=rio.band(dst, 1),
        dst_transform=dst_transform,
        dst_crs=reproj_crs,
        resampling=Resampling.bilinear
    )


